**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [4]:
FD_REG=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
ls -lh ${FD_REG}/${FN_REG}
cat ${FD_REG}/${FN_REG} | wc -l
echo +++++++++++++++++++++++++++++++++++
head ${FD_REG}/${FN_REG}
echo +++++++++++++++++++++++++++++++++++
tail ${FD_REG}/${FN_REG}

-rw-rw-r-- 1 kk319 reddylab 5.7M Oct 17 05:28 /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
247520
+++++++++++++++++++++++++++++++++++
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613
chr1	30803	31072
chr1	101603	101849
chr1	115411	115986
chr1	118518	118743
chr1	136071	137429
+++++++++++++++++++++++++++++++++++
chrX	155966947	155967395
chrX	155971561	155972273
chrX	155986274	155987619
chrX	155997267	155998045
chrX	156000382	156003205
chrX	156009687	156010227
chrX	156016391	156016836
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


In [8]:
ls ${FD_RES}/results

A001_K562_WSTARRseq  KS91_K562_ASTARRseq  Tewhey_K562_TileMPRA


In [12]:
PREFIX=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${PREFIX}/fragment_count

echo ${FD_INP}
ls ${FD_INP}/*.bed.gz | xargs -n 1 basename

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count
KS91_K562_ASTARRseq.Input.rep1.FADS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep1.GATA1.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep1.MYC.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.FADS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.GATA1.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.MYC.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.FADS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.GATA1.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.MYC.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.FADS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.GATA1.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.MYC.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.FADS.unstranded

In [14]:
FP_INPS=($(ls ${FD_INP}/*.bed.gz))

echo Total: ${#FP_INPS[@]}

Total: 40


In [26]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.%a.txt \
    --array 0-39 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output directory
PREFIX=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${PREFIX}/fragment_count
FD_OUT=${FD_RES}/results/${PREFIX}/coverage_astarrseq_peak_macs_input

FP_INPS=($(ls ${FD_INP}/*.bed.gz))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}

FD_REG=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_REG=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
zcat  ${FP_INP} | head -n 3 
echo

echo "Input  file: " ${FD_REG}/${FN_REG}
echo
echo "Show the first few lines of the input file"
cat  ${FD_REG}/${FN_REG} | head -n 3 
echo
echo "Show the last few lines of the input file"
cat  ${FD_REG}/${FN_REG} | tail -n 3 
echo

### execute
bedtools map \
    -a ${FD_REG}/${FN_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FD_OUT}/${FN_OUT}

### Show I/O file
echo
echo "Output file: " ${FD_OUT}/${FN_OUT}
echo
echo "Show the first few lines of the output file"
zcat ${FD_OUT}/${FN_OUT} | head -n 3
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29423303


In [27]:
ls ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.*.txt

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.0.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.10.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.11.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.12.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.13.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.14.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.15.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.16.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astarr.17.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/coverage_astarr_peak_macs_input_astar

In [28]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.0.txt

Hostname:           x1-01-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep1.FADS.unstranded.bed.gz

Show the first few lines of the input file
chr11	61554341	61554829	chr11_61554341_61554829	1	.
chr11	61554409	61554803	chr11_61554409_61554803	1	.
chr11	61554409	61555000	chr11_61554409_61555000	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRse

In [29]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.3.txt

Hostname:           x1-02-1.genome.duke.edu
Slurm Array Index:  3
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10143	chr1_10001_10143	1	.
chr1	10002	10118	chr1_10002_10118	1	.
chr1	10002	10131	chr1_10002_10131	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz

Show th

In [31]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.7.txt

Hostname:           x1-02-3.genome.duke.edu
Slurm Array Index:  7
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10233	chr1_10001_10233	1	.
chr1	10001	10351	chr1_10001_10351	1	.
chr1	10002	10119	chr1_10002_10119	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz

Show th

In [32]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.11.txt

Hostname:           x1-02-2.genome.duke.edu
Slurm Array Index:  11
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10203	chr1_10001_10203	1	.
chr1	10002	10113	chr1_10002_10113	1	.
chr1	10002	10131	chr1_10002_10131	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz

Show t

In [33]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.15.txt

Hostname:           x1-02-2.genome.duke.edu
Slurm Array Index:  15
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10125	chr1_10001_10125	1	.
chr1	10001	10326	chr1_10001_10326	1	.
chr1	10002	10119	chr1_10002_10119	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz

Show t

In [34]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.19.txt

Hostname:           x1-02-4.genome.duke.edu
Slurm Array Index:  19
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10119	chr1_10001_10119	1	.
chr1	10001	10131	chr1_10001_10131	1	.
chr1	10001	10215	chr1_10001_10215	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz

Show t

In [35]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.23.txt

Hostname:           x1-03-1.genome.duke.edu
Slurm Array Index:  23
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10001	10119	chr1_10001_10119	1	.
chr1	10001	10143	chr1_10001_10143	1	.
chr1	10002	10119	chr1_10002_10119	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz

Show t

In [36]:
cat ${FD_LOG}/coverage_astarr_peak_macs_input_astarr.27.txt

Hostname:           x1-03-2.genome.duke.edu
Slurm Array Index:  27
Time Stamp:         12-11-22+13:49:37

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Output.rep1.WGS.unstranded.bed.gz

Show the first few lines of the input file
chr1	10057	10233	chr1_10057_10233	2	.
chr1	10188	10442	chr1_10188_10442	1	.
chr1	10285	10349	chr1_10285_10349	1	.

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

Show the first few lines of the input file
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477

Show the last few lines of the input file
chrX	156024950	156025593
chrX	156030187	156030740
chrY	11215098	11215422


Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/KS91_K562_ASTARRseq.Output.rep1.WGS.unstranded.bed.gz

Show